In [1]:
# Downloading the dataset: Tiny shakespeare
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-03 08:10:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2023-10-03 08:10:41 (116 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# reading the dataset
with open("input.txt", "r", encoding = "utf-8") as f:
  text = f.read()

In [3]:
print("length of characters in dataset: ", len(text))

length of characters in dataset:  1115394


In [41]:
print(text[:2000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# unique characters that occur in the text
chars = sorted(list(set(text))) # using set constructor will help us to get the set of all the characters in the text and creating a list of that
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)

# if you highlight the text, you will notice you have a space character as well.
# There are 65 characters in total including a space character. These 65 characters will be part of our vocabulary.

# This is a simple tokenier and it is called as Character Level Tokeniser.
# Although it is simple but because of limited charaters/tokens we get a long list of integers as a result.


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# Developing a strategy to tokenize the input text.

# Tokenizing basically means converting a raw text (string) to some sequence of integers

In [7]:
# creating a mapping from characters to integers and reverse mapping || here we are building both encoder and the decoder.

stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # here we tranlating all the characters individually
decode = lambda l: "".join([itos[i] for i in l]) # decoding it back using reverse mapping

print(encode("hi there"))
print(decode(encode("hi there"))) # and this is what is reverse mapping

# The list in the output [46, 47, 1...] is what we get after encoding "hii there". When we encode (an arbitrary text), we will receive a list of integers that represents
# the string, we will also do the reverse mapping where we will take the list of integers and convert it back to string


# How this works? We iterate all the characters and create a lookup table (line 3) to convert character to int and vice-versa (line 4), we do encoding of the string (line 8)
# and decoding (line 6). This is one of many possible encodings. There are other schemas out there in practice, for eg: Google uses "sentencePiece" and in this you encode
# sub-words (not the entire word or the characters). OpenAI uses "tiktoken" that is a fast BPE (Byte Pair Encoding) tokeniser and the same tokeniser is used  by GPT. It has
# 50257 characters/tokens unlike the one we are using which as 65 tokens/characters.

# In a way, there is a trade-off between the code book size and sequence lengths, so you can have very long sequence of integers with small vocabulary or we can have short
# sequence of integers with very large vocabulary.

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [8]:
# Since now we have a tokeniser, we can tokenize the entire set of shakespeare.
# We will now encode the entire text dataset and store it into a torch.Tensor
import torch # From torch library, we are using torch.tensor module
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

# here we are taking the entire text, encode it and wrap it into a torch.tensor

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# splitting the data into train and validation split

n = int(0.9 * len(data)) # first 90% will be will used for train the Transformer and 10% for validation data.
train_data = data[:n] # training set
val_data = data[n:] # testing set

In [10]:
# Important: We will not be feeding the entire text to transformers all at once, that would be computationally expensive.
# when we train transformers, we train them on chunks of dataset.. ie we sample random chunks out of the training set and train them
# one chunk at a time.

In [11]:
# the length of the chunk is called block size

block_size = 8
train_data[:block_size + 1]

# output are the first nine characters in the training set. We will train it at every postion, in the context of 18, 47 comes next, for (18 + 45) 56 come next, in the
# context of (18, 47, 56) 57 come next

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size] # x are the inputs to the transformer, it will be the first block size characters
y = train_data[1 : block_size + 1] # y will be the next block size characters basically its offset by 1 because y are the targets for each position in the input
for t in range(block_size): # iterating over the block size of 8
  context = x[:t + 1] # all the characters in x up to t (including t)
  target = y[t] # target is always the t character
  print(f"When the input is {context}, the target is: {target}")



# Important thing to note here is that the output shows transformer getting trained on eight examples here, making sure it understand the "context" between all the integers
# from as little as one character to all the way to a block size.
# and that is why it is carrying forward the previous integer to the new integer. basically, we are making the transformer n/w used to seeing context all the way from as
# one to all way to block_size (block_size = [18, 47, 56, 57, 58,  1, 15, 47]) so that it can predict next character

When the input is tensor([18]), the target is: 47
When the input is tensor([18, 47]), the target is: 56
When the input is tensor([18, 47, 56]), the target is: 57
When the input is tensor([18, 47, 56, 57]), the target is: 58
When the input is tensor([18, 47, 56, 57, 58]), the target is: 1
When the input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [13]:
# Above code is the time dimension of the tensors that are we are feeding to the transformer, there is one more dimension to care about: batch_dimension

In [14]:
# here we are going to start sampling random locations in the dataset to pull chunks from, we are setting the seed in the random number generator.

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences we will process in parallel ie every forward backward pass in the transformer
block_size = 8 # what is the maximum context lenght for predictions


def get_batch(split):
# generating a small batch of data of inputs x and targets y
  data = train_data if split == "train" else val_data # if the split is a training split, then we will look at train data otherwise val_data and that will give us the data array
  ix = torch.randint(len(data) - block_size, (batch_size,)) # here we are generating random positions, since our batch_size is 4, ix will consist of four numbers randomly
# generated b/w 0 and len(data) - block_size
  x = torch.stack([data[i:i+block_size] for i in ix]) # x is the firt block size characters starting at i || We are going to get chunks for every one of integers of I in ix
# torch.stack will help us to take all one dimensional tensors and stack them up at rows. So basically, they all become a row in a 4 by 8 tensor
  y = torch.stack([data[i+1:i+block_size + 1] for i in ix]) # y is just ofset of 1 or + 1 of x [just a (+1) iteration of i]
  return x, y


xb, yb = get_batch("train") # sampling a batch on xb and yb. ||
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size): # batch dimension
  for t in range(block_size):# time dimension
    context = xb[b, :t + 1]
    target = yb[b, t]
    print(f"when the input is {context.tolist()} the target is: {target}")

# Input x is the 4 by 8 tensor ie 4 rows of 8 columns and each one of these is a chunk of the training set
# Targets are in the associated array Y and they will come through the Transformer all the way at the end to create the loss function, basically they will give us the correct
# answer for every single position inside x.

# inputs are x and targets are y
# So below are the 32 independent examples packed in to a single batch of input x and the desired targets are in y.


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when the input is [24] the target is: 43
when the input is [24, 43] the target is: 58
when the input is [24, 43, 58] the target is: 5
when the input is [24, 43, 58, 5] the target is: 57
when the input is [24, 43, 58, 5, 57] the target is: 1
when the input is [24, 43, 58, 5, 57, 1] the target is: 46
when the input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
when the input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
when the input is [44] the target is: 53
when the input is [44, 53] the target is: 56
when the input is [44, 53, 56] the target is: 1
when the input is [44, 53, 56, 1] the 

In [15]:
print(xb) # output is the batch of input that we would like to the transformers basically feeding it to the neural networks.

# we will start with the simplest possible neural network which is called as bigram language model


tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [19]:
# to implement the bigram LM we will use pytorch module

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): # Constructing a BigramLanguageModel which is a subclass of nn.Module

  def __init__(self, vocab_size):
    super().__init__() # each token reads off the logits for the next token from a look up table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # here we are creating a token_embedding_table that will be equivalent to the size of vocab_size
# nn.embedding is a very thin wrapper around tensor of shape for both (vocab_size by vocab_size)

  def forward(self, idx, targets = None): # idx = inputs x || passing idx into token embedding table
      logits = self.token_embedding_table(idx) # (B, T, C)  # pytorch will arrange all the calculation into (batch by time by channel) tensor || in this scenario batch is 4,
# time is 8 and c is the channel or vocab_size which is 65. So basically we are going to pluck out all the rows and arrange them in a B by T by C and we are going to interpret
# this as logits which are basically the scores for the next character in the sequence.
# here we are basically predicting what comes next based on the individual identity of a single token
      if  targets is None:
          loss = None
      else:
          B, T, C = logits.shape  # giving names to the dimensions
          logits = logits.view(B * T, C) # we chnage B, T, C (which was a 3 dimensional arary to a two dimensional array as pytorch expects the channels to be 2nd in numbers)
          targets = targets.view(B*T)
          loss = F.cross_entropy(logits, targets) # A good way to measure a loss or quality of the prediction is to use negative log likelyhood loss which is also implement in
    # pytorch under the name cross entropy.

      return logits, loss

# When we are passing the idx into the token_embedding_table every single integer will refer nn.embedding table and will pluck out a row of that embedding table corresponding
# to its index. for eg. 24 will go to the embedding table and will pluck out the 24th row, 43rd will pluck out 43 and so on

# Generating function for the model
# The job of generator is to take B, T (B by T) and make it B by T + 1,  B by T + 2..as many as we want

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, idx_next), dim = 1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # passing the inputs and the targets
print(logits.shape)
print(loss)
# The output that we are getting is scores or logits for every 4 by 8 positions

# now that we are able to make predictions what comes next we would now evaluate the loss function
# In the output we can see that the loss is 4.8786
# Since we have 65 vocabulary elements, we can calculate loss manually: -ln(1/65) = 4.17

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))




torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [20]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [21]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.587916374206543


In [22]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?


## The mathematical trick in self-attention

In [23]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [25]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [26]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [27]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [28]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [29]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:

* Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
* Each example across batch dimension is of course processed completely independently and never "talk" to each other
* In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
* "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
* "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [30]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var()

tensor(1.0449)

In [32]:
q.var()

tensor(1.0700)

In [33]:
wei.var()

tensor(1.0918)

In [34]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [35]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [36]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [37]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [38]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [39]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4196, val loss 2.4338
step 400: train loss 2.3503, val loss 2.3565
step 500: train loss 2.2966, val loss 2.3129
step 600: train loss 2.2410, val loss 2.2500
step 700: train loss 2.2051, val loss 2.2191
step 800: train loss 2.1640, val loss 2.1874
step 900: train loss 2.1251, val loss 2.1515
step 1000: train loss 2.1023, val loss 2.1291
step 1100: train loss 2.0699, val loss 2.1192
step 1200: train loss 2.0375, val loss 2.0797
step 1300: train loss 2.0259, val loss 2.0647
step 1400: train loss 1.9924, val loss 2.0362
step 1500: train loss 1.9700, val loss 2.0304
step 1600: train loss 1.9631, val loss 2.0476
step 1700: train loss 1.9412, val loss 2.0131
step 1800: train loss 1.9097, val loss 1.9960
step 1900: train loss 1.9101, val loss 1.9882
step 2000: train loss 1.8867, val loss 1.9976
step 2100: train loss 1.